<a href="https://colab.research.google.com/github/MasaHirai-Jinen/Medical_Institution_db_colab/blob/main/%E5%8C%BB%E7%99%82%E6%A9%9F%E9%96%A2%E5%8F%97%E7%90%86%E5%B1%8A%E9%A0%85%E7%9B%AE%E5%88%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

zip_file_path = "jmidb-bucket.zip"
file_id = '1kGQtfh7ZZDHLeK-0_OpFubKod06zb9ZL' # https://drive.google.com/file/d/1kGQtfh7ZZDHLeK-0_OpFubKod06zb9ZL/view?usp=drive_link
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(zip_file_path)

few_docor_path = "医師少数区域とスポットのICU.xlsx"
file_id = '1dxAQTJ_0my7uq4DRW-MtZErJ1PA4O_wP' # https://docs.google.com/spreadsheets/d/1dxAQTJ_0my7uq4DRW-MtZErJ1PA4O_wP/edit?usp=drive_link&ouid=117477706610962127029&rtpof=true&sd=true
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(few_docor_path)

### 共有method

In [2]:
import pandas as pd
import re

In [3]:
def print_dataframe_info(df:pd.DataFrame):
    print(df.info())
    print('-'*100)
    display(df)

In [9]:
import zipfile
import io
from datetime import datetime
from typing import List, Tuple


def extract_files_from_zip (zip_filepath, code_string) -> List[Tuple[datetime, str, io.BytesIO]]:
    file_list = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for info in zip_ref.infolist():
            l = info.filename.split('/')
            if (code_string in l) and (not info.is_dir()):
                i = l.index(code_string)
                category, kikan, download_date, *filename = l[i:]
                download_date = datetime.strptime(download_date, '%Y-%m-%d')
                with zip_ref.open(info) as file:
                    buffer = io.BytesIO(file.read())
                file_list.append((download_date, info.filename, buffer))

    file_list.sort(key=lambda x: x[0])
    return file_list

In [5]:
import pandas as pd
from typing import NamedTuple

TodofukenType = NamedTuple('TodofukenType', [('id', int), ('name', str)])

class TodofukenDataFrame:
    """
    都道府県データを保持するクラス

    Attributes:
        df (pd.DataFrame): 都道府県データ

    Examples:
        TODOFUKEN = TodofukenDataFrame()
        string = "ｘｘｘｘｘｘｘ kumamoto みえｘｘｘｘｘｘｘｘｘ"
        TODOFUKEN.find_todofuken_from_string(string) # -> TodofukenType(id=24, name='三重県')
    """
    __df: pd.DataFrame = pd.DataFrame(
            [[1,"北海道","北海道", "ほっかいどう", "hokkaido"],
            [2,"青森県","青森","あおもり", "aomori"],
            [3,"岩手県","岩手","いしかわ", "iwate"],
            [4,"宮城県","宮城","みやぎ", "miyagi"],
            [5,"秋田県","秋田","あきた", "akita"],
            [6,"山形県","山形","やまがた", "yamagata"],
            [7,"福島県","福島","ふくい", "fukushima"],
            [8,"茨城県","茨城","いばらき", "ibaraki"],
            [9,"栃木県","栃木","とちぎ", "tochigi"],
            [10,"群馬県","群馬","ぐんま", "gunma"],
            [11,"埼玉県","埼玉","さいたま", "saitama"],
            [12,"千葉県","千葉","ちば", "chiba"],
            [13,"東京都","東京","とうきょう", "tokyo"],
            [14,"神奈川県","神奈川","かながわ", "kanagawa"],
            [15,"新潟県","新潟","にいがた", "niigata"],
            [16,"富山県","富山","とやま", "toyama"],
            [17,"石川県","石川","いしかわ", "ishikawa"],
            [18,"福井県","福井","ふくい", "fukui"],
            [19,"山梨県","山梨","やまなし", "yamanashi"],
            [20,"長野県","長野","ながの", "nagano"],
            [21,"岐阜県","岐阜","ぎふ", "gifu"],
            [22,"静岡県","静岡","しずおか", "shizuoka"],
            [23,"愛知県","愛知","あいち", "aichi"],
            [24,"三重県","三重","みえ", "mie"],
            [25,"滋賀県","滋賀","しが", "shiga"],
            [26,"京都府","京都","きょうと", "kyoto"],
            [27,"大阪府","大阪","おおさか", "osaka"],
            [28,"兵庫県","兵庫","ひょうご", "hyogo"],
            [29,"奈良県","奈良","なら", "nara"],
            [30,"和歌山県","和歌山","わかやま", "wakayama"],
            [31,"鳥取県","鳥取","とっとり", "tottori"],
            [32,"島根県","島根", "しまね", "shimane"],
            [33,"岡山県","岡山","おかやま", "okayama"],
            [34,"広島県","広島","ひろしま", "hiroshima"],
            [35,"山口県","山口","やまぐち", "yamaguchi"],
            [36,"徳島県","徳島","とくしま", "tokushima"],
            [37,"香川県","香川","かがわ", "kagawa"],
            [38,"愛媛県","愛媛","えひめ", "ehime"],
            [39,"高知県","高知","こうち", "kochi"],
            [40,"福岡県","福岡","ふくおか", "fukuoka"],
            [41,"佐賀県","佐賀","さが", "saga"],
            [42,"長崎県","長崎","ながさき", "nagasaki"],
            [43,"熊本県","熊本","くまもと", "kumamoto"],
            [44,"大分県","大分","おおいた", "oita"],
            [45,"宮崎県","宮崎","みやざき", "miyazaki"],
            [46,"鹿児島県","鹿児島","かごしま", "kagoshima"],
            [47,"沖縄県","沖縄","おきなわ", "okinawa"]]
            , columns=["都道府県ID", "都道府県名", "漢字", "ひらがな", "英字"]
        )

    @property
    def df(self) -> pd.DataFrame:
        return self.__df

    def find_todofuken_from_string(self, string:str) -> TodofukenType|None:
        """
        都道府県名を検索する.
        注意：idが小さいものから順に検索される.

        Args:
            string (str): 検索する文字列

        Returns:
            TodofukenType|None: 検索結果
        """
        string = string.lower()
        mask = self.df[['漢字', 'ひらがな', '英字']].apply(lambda row: any(s in string for s in row), axis=1)
        if mask.any():
            row = self.df[mask].iloc[0]
            return TodofukenType(id = row["都道府県ID"], name = row["都道府県名"])
        return None

TODOFUKEN = TodofukenDataFrame()

In [6]:
import pandas as pd
from typing import NamedTuple

SyubetuType = NamedTuple('SyubetuType', [('id', int), ('name', str)])

class SyubetuData:
    __df: pd.DataFrame = pd.DataFrame(
        [[1, "医科"],
        [3, "歯科"],
        [4, "薬局"],
        ],
        columns=["種別ID", "種別名"]
    )

    @property
    def df(self) -> pd.DataFrame:
        return self.__df

    def find_syubetu_from_id(self, id:int) -> SyubetuType|None:
        if id in self.__df["種別ID"].values:
            return SyubetuType(id=id, name=self.__df[self.__df["種別ID"] == id]["種別名"].values[0])
        return None

    def find_syubetu_from_name(self, name:str) -> SyubetuType|None:
        if name in self.__df["種別名"].values:
            return SyubetuType(id=self.__df[self.__df["種別名"] == name]["種別ID"].values[0], name=name)
        return None

SYUBETU = SyubetuData()

In [7]:
from datetime import datetime
import re

def wareki_to_seireki(input_string:str, format_string:str) -> datetime:
    """
    和暦を西暦に変換する

    %g: 年号
    %e: 年
    %m: 月
    %d: 日

    Args:
        input_string  (str): 変換する文字列
        format_string (str): 変換する文字列のフォーマット

    Returns:
        datetime: 変換後の年月日

    Usage Example:
        wareki_to_seireki('昭和元年2月28日', '%g%e年%m月%d日') #-> datetime(1926, 2, 28, 0, 0)
    """
    ERA_DICT = {
        '令和': datetime(2019, 5, 1),
        '平成': datetime(1989, 1, 8),
        '昭和': datetime(1926, 12, 25),
        '大正': datetime(1912, 7, 30),
        '明治': datetime(1868, 1, 25),
    }

    def sort_ear_year(m):
        """
        年号と年をソートする
        return: (年号, 年)
        """
        if m.group(1) in ERA_DICT.keys():
            return m.group(1), m.group(2)
        else:
            return m.group(2), m.group(1)

    # 正規表現のパターンを作成
    pattern = format_string.replace(r'%g', f"({'|'.join(ERA_DICT.keys())})")
    pattern = pattern.replace(r'%e', r'(\d{1,2}|元)')
    pattern = pattern.replace(r'%m', r'\d{1,2}')
    pattern = pattern.replace(r'%d', r'\d{1,2}')

    # 正規表現でマッチ
    match = re.match(pattern, input_string)

    if match is None:
        raise ValueError(f"time data '{input_string}' does not match format '{format_string}'")

    era_name, ear_year = sort_ear_year(match)

    if ear_year == '元':
        ear_year = 0
    else:
        ear_year = int(ear_year)-1
        if ear_year < 0:
            raise ValueError(f"time data '{input_string}' does not match format '{format_string}'")
    year = ERA_DICT[era_name].year + ear_year

    # 新しいformat_stringを作成
    new_format_string = format_string.replace('%g', '').replace('%e', '')
    new_format_string+= "-%Y"

    # マッチした個所を削除したinput_stringを作成
    new_string = match.group(0)

    start , end = match.span(1)
    new_string = new_string[:start] + new_string[end:]
    i = end - start

    start , end = match.span(2)
    new_string = new_string[:start-i] + new_string[end-i:]

    new_string+= f"-{year}"

    return datetime.strptime(new_string, new_format_string)

### 個別

In [11]:
file_list = extract_files_from_zip(zip_file_path, 'accept_eath')
file_list[:5]

[(datetime.datetime(2024, 5, 20, 0, 0),
  'content/jmidb-bucket/accept_eath/kyushu/2024-05-20/r6_05_koumoku_yakkyoku_01_02.xlsx',
  <_io.BytesIO at 0x7ed1d0817c40>),
 (datetime.datetime(2024, 5, 20, 0, 0),
  'content/jmidb-bucket/accept_eath/kyushu/2024-05-20/r6_05_koumoku_ika_b_02.xlsx',
  <_io.BytesIO at 0x7ed1d0817c90>),
 (datetime.datetime(2024, 5, 20, 0, 0),
  'content/jmidb-bucket/accept_eath/kyushu/2024-05-20/r6_05_koumoku_shika_02.xlsx',
  <_io.BytesIO at 0x7ed1d0817d30>),
 (datetime.datetime(2024, 5, 20, 0, 0),
  'content/jmidb-bucket/accept_eath/kyushu/2024-05-20/r6_05_koumoku_ika_c_02.xlsx',
  <_io.BytesIO at 0x7ed1d0817d80>),
 (datetime.datetime(2024, 5, 20, 0, 0),
  'content/jmidb-bucket/accept_eath/kyushu/2024-05-20/r6_05_koumoku_yakkyoku_03_02.xlsx',
  <_io.BytesIO at 0x7ed1d0817dd0>)]

In [12]:
# 特定集中治療室管理料があるファイルのインデックス
[i for i, file_item in enumerate(file_list) if 'tokutei1' in file_item[1]]

[36, 261, 551, 751, 955, 1085]

In [13]:
download_date, filename, buffer = file_list[36]
dfs = pd.read_excel(buffer, sheet_name=None, dtype=str)

In [14]:
def download_excel(buffer):
    # bufferをローカルにdownload
    from google.colab import files
    buffer.seek(0)
    temp_file_name = "temp_file.xlsx"
    with open(temp_file_name, 'wb') as f:
        f.write(buffer.read())

    files.download(temp_file_name)

# download_excel(buffer)

In [15]:
sheet_name,df = list(dfs.items())[0]

def _crean_excel(df):
    df = df.copy()
    df.dropna(axis=0, how='all', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

df = _crean_excel(df)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,[令和 6年 5月 1日 現在 医科] 令和６年５月１日作成,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）
2,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,NaN,NaN
3,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,NaN,早期離床・リハビリテーション加算
4,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,当該治療室の病床数：,16床


In [16]:
# メタ情報ごとに分割
def _split_excel(df):
    df = df.copy()
    split_index = df.index[df.iloc[:,0].str.contains(r'\[.*\]', regex=True, na=False)]
    tantai_dfs = []
    for i, index in enumerate(split_index):
        block_df = df.iloc[index:split_index[i+1]].copy() if i < len(split_index)-1 else df.iloc[index:].copy()
        tantai_dfs.append(block_df)
    return tantai_dfs

tantai_dfs = _split_excel(df)
tantai_dfs[2].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
133,[令和 6年 5月 1日 現在 医科] 令和６年５月１日作成,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）
135,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,0774-20-1111,0774-20-2336,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,NaN,NaN
136,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,0774-20-1111,0774-20-2336,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,当該治療室の病床数：,6床
137,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,0774-20-1111,0774-20-2336,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,救命救急センターに係る事項：,高度救命救急センターである


In [17]:
# メタ情報とデータの分割
tantai_df = tantai_dfs[2]

def _split_meta_and_data(df):
    df = df.copy()
    i = (df.iloc[:,0]=='項番	').idxmax()
    meta_df = df.loc[:i].copy()
    data = df.loc[i+1:].copy()
    return meta_df, data

meta_df, data = _split_meta_and_data(tantai_df)
meta_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
133,[令和 6年 5月 1日 現在 医科] 令和６年５月１日作成,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# メタデータの加工

def _extract_meta_info(df):
    df = df.copy()
    df.dropna(axis=0, how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)

    meta_info = df.apply(lambda x: x.str.cat(sep='\t'), axis=1)
    meta_info = meta_info.str.cat(sep='\n')

    def _extract_date_and_syubetu(meta_info):
        # 種別の抽出
        syubetu = None
        as_of_date = None
        created_date = None

        pattern = re.compile(r'\[(.*)現在\s(.*)\](.*)作成')
        match_ = re.search(pattern, meta_info)
        if match_:
            as_of_date, syubetu_name, created_date = match_.groups()
            syubetu_name = syubetu_name.strip()
            syubetu = SYUBETU.find_syubetu_from_name(syubetu_name)
        return syubetu, as_of_date, created_date

    syubetu, as_of_date, created_date = _extract_date_and_syubetu(meta_info)

    return meta_info, syubetu, as_of_date, created_date

meta_info, syubetu, as_of_date, created_date = _extract_meta_info(meta_df)
print(meta_info)
print(syubetu, as_of_date, created_date)

[令和 6年 5月 1日 現在　医科]　令和６年５月１日作成
SyubetuType(id=1, name='医科') 令和 6年 5月 1日  　令和６年５月１日


In [19]:
# データの加工
def _clean_data(df, as_of_date, created_date, syubetu):
    df = df.copy()

    df.columns = df.iloc[0]
    df = df[1:]
    df.reset_index(drop=True, inplace=True)
    df['基準日'] = as_of_date
    df['作成日'] = created_date
    df['点数区分コード'] = syubetu.id if syubetu else None
    df = df.astype({
        '都道府県コード': pd.Int64Dtype(),
        '点数区分コード': pd.Int64Dtype(),
        '医療機関番号': pd.Int64Dtype(),
        '併設医療機関番号': pd.Int64Dtype()})

    return df

data_df = _clean_data(data, as_of_date, created_date, syubetu)
print_dataframe_info(data_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   項番             224 non-null    object
 1   都道府県コード        224 non-null    Int64 
 2   都道府県名          224 non-null    object
 3   受理届出名称         224 non-null    object
 4   医療機関番号         224 non-null    Int64 
 5   併設医療機関番号       174 non-null    Int64 
 6   医療機関記号番号       0 non-null      object
 7   医療機関名称         224 non-null    object
 8   医療機関所在地（郵便番号）  224 non-null    object
 9   医療機関所在地（住所）    224 non-null    object
 10  電話番号           224 non-null    object
 11  FAX番号          126 non-null    object
 12  病床数            224 non-null    object
 13  受理記号           224 non-null    object
 14  受理番号           224 non-null    object
 15  算定開始年月日        224 non-null    object
 16  個別有効開始年月日      0 non-null      object
 17  備考（見出し）        82 non-null     object
 18  備考（データ）        176 non-null   

134,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,基準日,作成日,点数区分コード
0,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,...,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,NaN,NaN,令和 6年 5月 1日,令和６年５月１日,1
1,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,...,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,当該治療室の病床数：,6床,令和 6年 5月 1日,令和６年５月１日,1
2,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,...,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,救命救急センターに係る事項：,高度救命救急センターである,令和 6年 5月 1日,令和６年５月１日,1
3,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,...,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,救命救急センターに係る事項：,充実段階がＳである,令和 6年 5月 1日,令和６年５月１日,1
4,1,26,京都府,救命救急入院料１,1203692,1263692,NaN,医療法人徳洲会 宇治徳洲会病院,611－0041,京都府宇治市槇島町石橋１４５番,...,一般 473\n一般（感染）\n 6,救１,第29号,令和 6年 4月 1日,NaN,NaN,当該保険医療機関内に専任の小児科医が常時配置されている,令和 6年 5月 1日,令和６年５月１日,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,8,26,京都府,脳卒中ケアユニット入院医療管理料,9700012,9760012,NaN,京都第二赤十字病院,602－8026,京都市上京区釜座通丸太町上ル春帯町３５５－５,...,一般 667,脳卒中ケア,第39号,令和 5年10月 1日,NaN,NaN,NaN,令和 6年 5月 1日,令和６年５月１日,1
220,8,26,京都府,脳卒中ケアユニット入院医療管理料,9700012,9760012,NaN,京都第二赤十字病院,602－8026,京都市上京区釜座通丸太町上ル春帯町３５５－５,...,一般 667,脳卒中ケア,第39号,令和 5年10月 1日,NaN,NaN,早期栄養介入管理加算,令和 6年 5月 1日,令和６年５月１日,1
221,8,26,京都府,脳卒中ケアユニット入院医療管理料,9700012,9760012,NaN,京都第二赤十字病院,602－8026,京都市上京区釜座通丸太町上ル春帯町３５５－５,...,一般 667,脳卒中ケア,第39号,令和 5年10月 1日,NaN,当該治療室の病床数:,6床,令和 6年 5月 1日,令和６年５月１日,1
222,9,26,京都府,脳卒中ケアユニット入院医療管理料,9900026,<NA>,NaN,独立行政法人国立病院機構舞鶴医療センター,625－8502,舞鶴市字行永２４１０,...,一般 279\n精神 120,脳卒中ケア,第24号,平成30年10月 1日,NaN,NaN,NaN,令和 6年 5月 1日,令和６年５月１日,1


In [20]:
# ベッド数の抽出
def _extract_bed_num(df):
    df = df.copy()
    groups = df.groupby(['医療機関番号', '都道府県コード', '受理届出名称','基準日'])
    bed_num_rows = []
    for (number, todofuken_num, juri, created_date), group_df in groups:
        # if not juri.startswith('特定集中治療室管理料'):
        #     continue
        bed_nums = group_df[group_df['備考（見出し）'].str.contains('病床数', na=False)]['備考（データ）']
        if not bed_nums.empty:
            bed_nums = bed_nums.str.extract(r'(\d+)').astype(int)
            bed_nums = bed_nums.iloc[:,0].values.tolist()
            for bed_num in bed_nums:
                bed_num_rows.append([number, todofuken_num, juri, created_date, bed_num])

    bed_num_df = pd.DataFrame(bed_num_rows, columns=['医療機関番号', '都道府県コード', '受理届出名称','基準日', 'ベッド数'])
    return bed_num_df

bed_num_df = _extract_bed_num(data_df)
print_dataframe_info(bed_num_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   医療機関番号   50 non-null     int64 
 1   都道府県コード  50 non-null     int64 
 2   受理届出名称   50 non-null     object
 3   基準日      50 non-null     object
 4   ベッド数     50 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.1+ KB
None
----------------------------------------------------------------------------------------------------


,医療機関番号,都道府県コード,受理届出名称,基準日,ベッド数
0,308625,26,ハイケアユニット入院医療管理料１,令和 6年 5月 1日,6
1,405124,26,ハイケアユニット入院医療管理料１,令和 6年 5月 1日,4
2,405124,26,ハイケアユニット入院医療管理料１,令和 6年 5月 1日,4
3,405124,26,特定集中治療室管理料１,令和 6年 5月 1日,8
4,405124,26,脳卒中ケアユニット入院医療管理料,令和 6年 5月 1日,6
5,707065,26,ハイケアユニット入院医療管理料１,令和 6年 5月 1日,12
6,903045,26,ハイケアユニット入院医療管理料２,令和 6年 5月 1日,12
7,903367,26,ハイケアユニット入院医療管理料１,令和 6年 5月 1日,4
8,903367,26,特定集中治療室管理料１,令和 6年 5月 1日,10
9,903367,26,脳卒中ケアユニット入院医療管理料,令和 6年 5月 1日,3


In [21]:
def _clean_data_df(df):
    df = df.copy()
    df.drop_duplicates(subset=['医療機関番号', '都道府県コード', '受理届出名称'], inplace=True)
    df.drop(columns=['項番','病床数','備考（見出し）','備考（データ）'], inplace=True)
    df.sort_values(by=['都道府県コード', '医療機関番号'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

resalt_df = _clean_data_df(data_df)
print_dataframe_info(resalt_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   都道府県コード        48 non-null     Int64 
 1   都道府県名          48 non-null     object
 2   受理届出名称         48 non-null     object
 3   医療機関番号         48 non-null     Int64 
 4   併設医療機関番号       36 non-null     Int64 
 5   医療機関記号番号       0 non-null      object
 6   医療機関名称         48 non-null     object
 7   医療機関所在地（郵便番号）  48 non-null     object
 8   医療機関所在地（住所）    48 non-null     object
 9   電話番号           48 non-null     object
 10  FAX番号          28 non-null     object
 11  受理記号           48 non-null     object
 12  受理番号           48 non-null     object
 13  算定開始年月日        48 non-null     object
 14  個別有効開始年月日      0 non-null      object
 15  基準日            48 non-null     object
 16  作成日            48 non-null     object
 17  点数区分コード        48 non-null     Int64 
dtypes: Int64(4), object(14)
memory u

134,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,受理記号,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード
0,26,京都府,ハイケアユニット入院医療管理料１,308625,<NA>,NaN,医療法人社団洛和会 洛和会丸太町病院,604－8401,京都市中京区聚楽廻松下町９番７,075-801-0351,075-822-9427,ハイケア１,第37号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
1,26,京都府,特定集中治療室管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,075-361-7602,集１,第37号,令和 4年10月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
2,26,京都府,ハイケアユニット入院医療管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,075-361-7602,ハイケア１,第38号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
3,26,京都府,脳卒中ケアユニット入院医療管理料,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,075-361-7602,脳卒中ケア,第35号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
4,26,京都府,ハイケアユニット入院医療管理料１,707065,767065,NaN,公益社団法人京都保健会京都民医連中央病院,616－8147,京都市右京区太秦土本町２番地１,075-861-2220,075-882-5781,ハイケア１,第41号,令和 4年 8月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
5,26,京都府,ハイケアユニット入院医療管理料２,903045,963045,NaN,蘇生会総合病院,612－8473,京都市伏見区下鳥羽広長町１０１番地,075-621-3101,075-612-5790,ハイケア２,第20号,令和 6年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
6,26,京都府,特定集中治療室管理料１,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,075-571-8877,集１,第41号,令和 5年 7月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
7,26,京都府,ハイケアユニット入院医療管理料１,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,075-571-8877,ハイケア１,第47号,令和 5年 9月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
8,26,京都府,脳卒中ケアユニット入院医療管理料,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,075-571-8877,脳卒中ケア,第36号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1
9,26,京都府,特定集中治療室管理料１,1100591,1160591,NaN,京都岡本記念病院,613－0034,京都府久世郡久御山町佐山西ノ口１００番地,0774-48-5500,0774-44-7159,集１,第35号,令和 4年10月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1


In [22]:
def _merge_bed_num(df, bed_num_df):
    return df.merge(bed_num_df, on=['医療機関番号', '都道府県コード', '受理届出名称', '基準日'], how='left').astype({'ベッド数': pd.Int64Dtype()})

all_df = _merge_bed_num(resalt_df, bed_num_df)
print_dataframe_info(all_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   都道府県コード        53 non-null     Int64 
 1   都道府県名          53 non-null     object
 2   受理届出名称         53 non-null     object
 3   医療機関番号         53 non-null     Int64 
 4   併設医療機関番号       39 non-null     Int64 
 5   医療機関記号番号       0 non-null      object
 6   医療機関名称         53 non-null     object
 7   医療機関所在地（郵便番号）  53 non-null     object
 8   医療機関所在地（住所）    53 non-null     object
 9   電話番号           53 non-null     object
 10  FAX番号          33 non-null     object
 11  受理記号           53 non-null     object
 12  受理番号           53 non-null     object
 13  算定開始年月日        53 non-null     object
 14  個別有効開始年月日      0 non-null      object
 15  基準日            53 non-null     object
 16  作成日            53 non-null     object
 17  点数区分コード        53 non-null     Int64 
 18  ベッド数           50 non-null     I

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,受理記号,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数
0,26,京都府,ハイケアユニット入院医療管理料１,308625,<NA>,NaN,医療法人社団洛和会 洛和会丸太町病院,604－8401,京都市中京区聚楽廻松下町９番７,075-801-0351,075-822-9427,ハイケア１,第37号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,6
1,26,京都府,特定集中治療室管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,075-361-7602,集１,第37号,令和 4年10月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,8
2,26,京都府,ハイケアユニット入院医療管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,075-361-7602,ハイケア１,第38号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,4
3,26,京都府,ハイケアユニット入院医療管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,075-361-7602,ハイケア１,第38号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,4
4,26,京都府,脳卒中ケアユニット入院医療管理料,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,075-361-7602,脳卒中ケア,第35号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,6
5,26,京都府,ハイケアユニット入院医療管理料１,707065,767065,NaN,公益社団法人京都保健会京都民医連中央病院,616－8147,京都市右京区太秦土本町２番地１,075-861-2220,075-882-5781,ハイケア１,第41号,令和 4年 8月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,12
6,26,京都府,ハイケアユニット入院医療管理料２,903045,963045,NaN,蘇生会総合病院,612－8473,京都市伏見区下鳥羽広長町１０１番地,075-621-3101,075-612-5790,ハイケア２,第20号,令和 6年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,12
7,26,京都府,特定集中治療室管理料１,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,075-571-8877,集１,第41号,令和 5年 7月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,10
8,26,京都府,ハイケアユニット入院医療管理料１,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,075-571-8877,ハイケア１,第47号,令和 5年 9月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,4
9,26,京都府,脳卒中ケアユニット入院医療管理料,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,075-571-8877,脳卒中ケア,第36号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,3


In [23]:
# 医師少数地域・スポット
def _create_few_doctors_df(xlsx_path):

    few_doctors_df = pd.read_excel(xlsx_path,dtype=str)
    few_doctors_df.set_index('#', drop=True, inplace=True)
    # indexがnanは削除
    few_doctors_df = few_doctors_df[few_doctors_df.index.notna()]
    # カラム名から※以降の文字列を削除
    few_doctors_df.columns = few_doctors_df.columns.str.replace(r'※.*', '', regex=True)
    # 医師少数区域列でnanをFalseそれ以外をTrueにする
    target_col = ['医師少数区域', '医師少数スポット','別表第六の二厚労大臣が定める地域']
    few_doctors_df[target_col] = few_doctors_df[target_col].notna()
    few_doctors_df['点数区分コード'] = 1
    few_doctors_df = few_doctors_df[['都道府県コード','医療機関番号','医師少数区域','医師少数スポット','別表第六の二厚労大臣が定める地域','点数区分コード']]
    few_doctors_df = few_doctors_df.astype({'都道府県コード': pd.Int64Dtype(), '医療機関番号': pd.Int64Dtype()})

    return few_doctors_df.copy()

few_doctors_df = _create_few_doctors_df(few_docor_path)
print_dataframe_info(few_doctors_df)
# few_doctors_df[['都道府県コード','点数区分コード','医療機関番号','医師少数区域','医師少数スポット','別表第六の二厚労大臣が定める地域']]

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 1 to 29
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   都道府県コード           29 non-null     Int64
 1   医療機関番号            29 non-null     Int64
 2   医師少数区域            29 non-null     bool 
 3   医師少数スポット          29 non-null     bool 
 4   別表第六の二厚労大臣が定める地域  29 non-null     bool 
 5   点数区分コード           29 non-null     int64
dtypes: Int64(2), bool(3), int64(1)
memory usage: 1.0+ KB
None
----------------------------------------------------------------------------------------------------


,都道府県コード,医療機関番号,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域,点数区分コード
#,,,,,,
1,2,315255,True,False,False,1
2,2,810339,True,False,True,1
3,6,812497,True,False,False,1
4,7,211772,True,False,False,1
5,7,210246,True,False,False,1
6,7,411497,True,False,False,1
7,8,210047,True,False,False,1
8,8,3811171,True,False,False,1
9,8,810663,True,False,False,1


In [24]:
def _merge_few_doctors_df(all_df, few_doctors_df):
    return_df = all_df.merge(few_doctors_df, on=['都道府県コード', '点数区分コード', '医療機関番号'], how='left')

    bool_cols = ['医師少数区域', '医師少数スポット','別表第六の二厚労大臣が定める地域']
    return_df.loc[:,bool_cols] = return_df[bool_cols].astype(float).fillna(0).astype(bool)
    return return_df

return_df = _merge_few_doctors_df(all_df, few_doctors_df)
print_dataframe_info(return_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   都道府県コード           53 non-null     Int64 
 1   都道府県名             53 non-null     object
 2   受理届出名称            53 non-null     object
 3   医療機関番号            53 non-null     Int64 
 4   併設医療機関番号          39 non-null     Int64 
 5   医療機関記号番号          0 non-null      object
 6   医療機関名称            53 non-null     object
 7   医療機関所在地（郵便番号）     53 non-null     object
 8   医療機関所在地（住所）       53 non-null     object
 9   電話番号              53 non-null     object
 10  FAX番号             33 non-null     object
 11  受理記号              53 non-null     object
 12  受理番号              53 non-null     object
 13  算定開始年月日           53 non-null     object
 14  個別有効開始年月日         0 non-null      object
 15  基準日               53 non-null     object
 16  作成日               53 non-null     object
 17  点数区分コード           

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域
0,26,京都府,ハイケアユニット入院医療管理料１,308625,<NA>,NaN,医療法人社団洛和会 洛和会丸太町病院,604－8401,京都市中京区聚楽廻松下町９番７,075-801-0351,...,第37号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,6,False,False,False
1,26,京都府,特定集中治療室管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,...,第37号,令和 4年10月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,8,False,False,False
2,26,京都府,ハイケアユニット入院医療管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,...,第38号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,4,False,False,False
3,26,京都府,ハイケアユニット入院医療管理料１,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,...,第38号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,4,False,False,False
4,26,京都府,脳卒中ケアユニット入院医療管理料,405124,<NA>,NaN,武田病院,600－8558,京都市下京区塩小路通西洞院東入東塩小路町８４１番地の５,075-361-1351,...,第35号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,6,False,False,False
5,26,京都府,ハイケアユニット入院医療管理料１,707065,767065,NaN,公益社団法人京都保健会京都民医連中央病院,616－8147,京都市右京区太秦土本町２番地１,075-861-2220,...,第41号,令和 4年 8月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,12,False,False,False
6,26,京都府,ハイケアユニット入院医療管理料２,903045,963045,NaN,蘇生会総合病院,612－8473,京都市伏見区下鳥羽広長町１０１番地,075-621-3101,...,第20号,令和 6年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,12,False,False,False
7,26,京都府,特定集中治療室管理料１,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,...,第41号,令和 5年 7月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,10,False,False,False
8,26,京都府,ハイケアユニット入院医療管理料１,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,...,第47号,令和 5年 9月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,4,False,False,False
9,26,京都府,脳卒中ケアユニット入院医療管理料,903367,963367,NaN,医療法人医仁会 武田総合病院,601－1495,京都市伏見区石田森南町２８－１,075-572-6331,...,第36号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,3,False,False,False


In [25]:
return_df[return_df[['医師少数区域', '医師少数スポット','別表第六の二厚労大臣が定める地域']].any(axis=1)]

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域
44,26,京都府,特定集中治療室管理料３,9700095,9760095,NaN,国家公務員共済組合連合会 舞鶴共済病院,625－8585,舞鶴市字浜１０３５,0773-62-2510,...,第86号,令和 5年 5月 1日,NaN,令和 6年 5月 1日,令和６年５月１日,1,10,False,True,False


### まとめ

In [26]:
from tqdm import tqdm

def main():
    file_list = extract_files_from_zip(zip_file_path, 'accept_eath')
    few_doctors_df = _create_few_doctors_df(few_docor_path)
    return_dfs = []
    for (download_date, filename, buffer) in tqdm(file_list):
        dfs = pd.read_excel(buffer, sheet_name=None, dtype=str)
        sheet_dfs = []

        for sheet_name, df in dfs.items():
            df = _crean_excel(df)
            tantai_dfs = _split_excel(df)
            for tantai_df in tantai_dfs:
                meta_df, data_df = _split_meta_and_data(tantai_df)
                meta_info, syubetu, as_of_date, created_date = _extract_meta_info(meta_df)
                cleaned_data_df = _clean_data(data_df, as_of_date, created_date, syubetu)
                bed_num_df = _extract_bed_num(cleaned_data_df)
                comped_data_df = _clean_data_df(cleaned_data_df)
                sheet_df = _merge_bed_num(comped_data_df, bed_num_df)

                sheet_dfs.append(sheet_df)
        file_df = pd.concat(sheet_dfs)

        file_df['ファイルパス'] = filename
        file_df['データ取得日'] = download_date
        return_df = _merge_few_doctors_df(file_df, few_doctors_df)
        return_dfs.append(return_df)
    return pd.concat(return_dfs)

main_df = main()
print_dataframe_info(main_df)

100%|██████████| 1091/1091 [16:22<00:00,  1.11it/s]


<class 'pandas.core.frame.DataFrame'>
Index: 904677 entries, 0 to 1287
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   都道府県コード           904677 non-null  Int64         
 1   都道府県名             904677 non-null  object        
 2   受理届出名称            904677 non-null  object        
 3   医療機関番号            904677 non-null  Int64         
 4   併設医療機関番号          75892 non-null   Int64         
 5   医療機関記号番号          208361 non-null  object        
 6   医療機関名称            904677 non-null  object        
 7   医療機関所在地（郵便番号）     904677 non-null  object        
 8   医療機関所在地（住所）       904677 non-null  object        
 9   電話番号              904677 non-null  object        
 10  FAX番号             724517 non-null  object        
 11  受理記号              904677 non-null  object        
 12  受理番号              904677 non-null  object        
 13  算定開始年月日           904677 non-null  object        
 14  個別有効開始年月日  

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,ファイルパス,データ取得日,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域
0,40,福岡県,後発医薬品調剤体制加算１,240006,<NA>,NaN,福薬局箱崎駅前店,812－0053,福岡市東区箱崎三丁目５番４２号１Ｆ,092-632-3210,...,NaN,令和 6年 5月 1日,令和６年５月１６日,4,<NA>,content/jmidb-bucket/accept_eath/kyushu/2024-0...,2024-05-20,False,False,False
1,40,福岡県,かかりつけ薬剤師指導料及びかかりつけ薬剤師包括管理料,240006,<NA>,NaN,福薬局箱崎駅前店,812－0053,福岡市東区箱崎三丁目５番４２号１Ｆ,092-632-3210,...,NaN,令和 6年 5月 1日,令和６年５月１６日,4,<NA>,content/jmidb-bucket/accept_eath/kyushu/2024-0...,2024-05-20,False,False,False
2,40,福岡県,在宅患者訪問薬剤管理指導料,240006,<NA>,NaN,福薬局箱崎駅前店,812－0053,福岡市東区箱崎三丁目５番４２号１Ｆ,092-632-3210,...,NaN,令和 6年 5月 1日,令和６年５月１６日,4,<NA>,content/jmidb-bucket/accept_eath/kyushu/2024-0...,2024-05-20,False,False,False
3,40,福岡県,かかりつけ薬剤師指導料及びかかりつけ薬剤師包括管理料,341267,<NA>,NaN,株式会社 十字堂薬局,812－0044,福岡県福岡市博多区千代４丁目３０－５,092-641-4115,...,NaN,令和 6年 5月 1日,令和６年５月１６日,4,<NA>,content/jmidb-bucket/accept_eath/kyushu/2024-0...,2024-05-20,False,False,False
4,40,福岡県,在宅患者訪問薬剤管理指導料,341267,<NA>,NaN,株式会社 十字堂薬局,812－0044,福岡県福岡市博多区千代４丁目３０－５,092-641-4115,...,NaN,令和 6年 5月 1日,令和６年５月１６日,4,<NA>,content/jmidb-bucket/accept_eath/kyushu/2024-0...,2024-05-20,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,30,和歌山県,短期滞在手術等基本料１,2310284,<NA>,新医1028,さとう眼科,647－0045,新宮市井の沢１２－６,0735-28-0310,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,4,content/jmidb-bucket/accept_eath/kinki/2024-11...,2024-11-06,False,False,False
1284,30,和歌山県,短期滞在手術等基本料１,2410803,<NA>,西医1080,上富田クリニック,649－2105,西牟婁郡上富田町朝来１４０７－１,0739-47-1100,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,<NA>,content/jmidb-bucket/accept_eath/kinki/2024-11...,2024-11-06,False,False,False
1285,30,和歌山県,短期滞在手術等基本料１,2510461,<NA>,東医1046,医療法人健佑会 串本有田病院,649－3514,東牟婁郡串本町有田４９９番地の１,0735-66-1021,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,4,content/jmidb-bucket/accept_eath/kinki/2024-11...,2024-11-06,False,False,False
1286,30,和歌山県,短期滞在手術等基本料１,2510479,<NA>,東医1047,串本リハビリテーションセンター,649－3503,東牟婁郡串本町串本２５９－６,0735-62-3600,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,<NA>,content/jmidb-bucket/accept_eath/kinki/2024-11...,2024-11-06,False,False,False


In [27]:
output_csv_path = 'main_df.csv'
main_df.to_csv(output_csv_path, index=False)

In [ ]:
# Fix: これじゃダメ。ベッド数が重複している可能性があるので、このロジックで重複を削除した後にベッド数をマージする必要がある
output_df = main_df.drop_duplicates(subset=['都道府県コード', '点数区分コード', '医療機関番号', '作成日', '受理記号']).sort_values(by=['都道府県コード', '点数区分コード', '医療機関番号', '作成日', '受理記号']).copy()
print_dataframe_info(output_df)

In [ ]:
sorted(output_df['受理届出名称'].unique())

In [ ]:
output_df['医療機関コード'] = output_df['都道府県コード'].astype(str).str.zfill(2)\
                              + output_df['点数区分コード'].astype(str).str.zfill(1)\
                              + output_df['医療機関番号'].astype(str).str.zfill(7)

fileter = [
 '救命救急入院料１',
 '救命救急入院料２',
 '救命救急入院料３',
 '救命救急入院料４',
 '特定集中治療室管理料１',
 '特定集中治療室管理料２',
 '特定集中治療室管理料３',
 '特定集中治療室管理料４',
 '特定集中治療室管理料５',
 '特定集中治療室管理料６',
 '小児特定集中治療室管理料',
 'ハイケアユニット入院医療管理料１',
 'ハイケアユニット入院医療管理料２',
 '脳卒中ケアユニット入院医療管理料'
]
output_df = output_df[output_df['受理届出名称'].isin(fileter)]

print_dataframe_info(output_df)

In [ ]:
# 作成日ごとにシートを分けてExcelファイルに抽出
output_excel_path = 'output_df.xlsx'

with pd.ExcelWriter(output_excel_path) as writer:
    for date, date_df in output_df.groupby('作成日'):
        date_df.to_excel(writer, sheet_name=date, index=False)

In [ ]:
from google.colab import files
files.download(output_excel_path)



**全column**
- 都道府県コード : int
- 都道府県名
- 点数区分コード : int
- 医療機関番号 : int
- 併設医療機関番号 : int
- 医療機関記号番号
- 医療機関名称
- 医療機関所在地（郵便番号）
- 医療機関所在地（住所）
- 電話番号
- FAX番号
- 医師少数地域 : bool
- 医師少数スポット : bool
- 別表第六の二厚労大臣が定める地域 : bool
- 受理記号
- 受理届出名称
- 受理番号
- 算定開始年月日
- 個別有効開始年月日
- ベッド数 : int
- 基準日：データの内容が何時時点のものか示す日付
- 作成日：厚生局がデータを作成した日付
- データ取得日：データをスクレイピングした日付
- ファイルパス


**DB**
- 医療機関
    - 都道府県コード `PK`
    - 都道府県名
    - 点数区分コード `PK`
    - 医療機関番号 `PK`
    - 併設医療機関番号
    - 医療機関記号番号
    - 医療機関名称
    - 医療機関所在地（郵便番号）
    - 医療機関所在地（住所）
    - 電話番号
    - FAX番号
    - 作成日 `PK`
    - 医師少数地域
    - 医師少数スポット

- 入院科料
    - 都道府県コード `FK` `PK`
    - 点数区分コード `FK` `PK`
    - 医療機関番号 `FK` `PK`
    - 作成日 `FK` `PK`
    - 受理記号 `PK`
    - 受理届出名称
    - 受理番号
    - 算定開始年月日
    - 個別有効開始年月日


- ベッド数
    - id `PK` `AUTO`
    - 都道府県コード `FK`
    - 点数区分コード `FK`
    - 医療機関番号 `FK`
    - 作成日 `FK`
    - 受理記号 `FK`
    - ベッド数

In [ ]:
# メタ情報の確認
import pandas as pd
import re
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

pattern = re.compile(r'\[(.*)現在\s(.*)\](.*)作成')

def process_file(file_item):
    download_date, filename, buffer = file_item
    dfs = pd.read_excel(buffer, sheet_name=None, dtype=str)
    for sheet_name, df in dfs.items():
        for tantai_df in _split_excel(df):
            meta_df, data = _split_meta_and_data(tantai_df)
            match_ = re.search(pattern, meta_df.iloc[0,0])
            if match_:
                as_of_date, syubetu_name, created_date = match_.groups()
                if '' in [as_of_date, syubetu_name, created_date]:
                    print(meta_df.iloc[0,0], filename)
                    print('-'*100)
                # print(as_of_date, syubetu_name, created_date)
            else:
                print(meta_df.iloc[0,0], filename)
                print('-'*100)

with ProcessPoolExecutor(max_workers=4) as executor:
    list(tqdm(executor.map(process_file, file_list), total=len(file_list)))

In [ ]:
column_names = pd.Index([
    "項番",
    "都道府県コード",
    "都道府県名",
    "受理届出名称",
    "医療機関番号",
    "併設医療機関番号",
    "医療機関記号番号",
    "医療機関名称",
    "医療機関所在地（郵便番号）",
    "医療機関所在地（住所）",
    "電話番号",
    "FAX番号",
    "病床数",
    "受理記号",
    "受理番号",
    "算定開始年月日",
    "個別有効開始年月日",
    "備考（見出し）",
    "備考（データ）"
])